In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_VISIBLE_DEVICES"]=''

import numpy as np
import tensorflow as tf

# import data

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_sequence = open_pickle('../data/imdb/X_tr_sample_original.pkl')
X_test_sequence = open_pickle('../data/imdb/X_te_sample_original.pkl')
y_train_target = open_pickle('../data/imdb/y_tr_target_original.pkl')
y_test_target = open_pickle('../data/imdb/y_te_target_original.pkl')

In [3]:
# X = open_pickle('../data/imdb_sequence/3000_one_hot/X_tr_seq_set.pkl')
# y = open_pickle('../data/imdb_sequence/3000_one_hot/y_tr_seq_set.pkl')

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3445910330077896616
]


### Preprocess

#### One Hot

In [5]:
from keras.preprocessing.text import text_to_word_sequence, one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS = 1000

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train_sequence)

num_encoder_tokens = tokenizer.num_words
num_decoder_tokens = tokenizer.num_words
max_encoder_seq_length = 81
max_decoder_seq_length = 81

# Input (X, y) needs to be raw sentences

def generate_one_hot(X, y, X_test=None, y_test=None):
    X_tr_padded = pad_sequences(tokenizer.texts_to_sequences(X), maxlen=81, padding='post', truncating='post')
    y_tr_padded = pad_sequences(tokenizer.texts_to_sequences(y), maxlen=5, padding='post', truncating='post')
    
    # Variable init
    encoder_input_data = np.zeros(
        (len(X), max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    decoder_input_data = np.zeros(
        (len(X), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')
    decoder_target_data = np.zeros(
        (len(y), max_decoder_seq_length, num_decoder_tokens),
        dtype='float32')

    # Generate sequence
    for i, (input_text, target_text) in enumerate(zip(X_tr_padded, y_tr_padded)):
        for t, word in enumerate(input_text):
            encoder_input_data[i, t, word] = 1.

        for t, word in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, word] = 1.

            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, word] = 1.
                
    if X_test is not None and y_test is not None:
        X_te_padded = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=81, padding='post', truncating='post')
        y_te_padded = pad_sequences(tokenizer.texts_to_sequences(y_test), maxlen=5, padding='post', truncating='post')
        
        return X_tr_padded, y_tr_padded, X_te_padded, y_te_padded, encoder_input_data, decoder_input_data, decoder_target_data
    else:
        return X_tr_padded, y_tr_padded, encoder_input_data, decoder_input_data, decoder_target_data

Using TensorFlow backend.


In [14]:
# X_tr_padded, y_tr_padded, encoder_input_data, decoder_input_data, decoder_target_data = generate_one_hot(X_train_sequence, y_train_target)

In [14]:
# X_train = {
#     'text': X_train_sequence,
#     'padded' : X_tr_padded
# }


# X_test = {
#     'text' : X_test_sequence,
#     'padded' : X_te_padded
# }

# y_train = {
#     'text': y_train_target,
#     'padded' : y_tr_padded
# }

# y_test = {
#     'text' : y_test_target,
#     'padded' : y_te_padded
# }

# X_tr_encoder_input = {
#     'encoder_input' : encoder_input_data
# }

# y_tr_decoder = {
#     'decoder_input' : decoder_input_data,
#     'decoder_target' : decoder_target_data
# }

In [15]:
# import pickle
# pickle.dump(X_train, open('../data/imdb_sequence/1000_one_hot/X_tr_seq_set.pkl','wb'))
# pickle.dump(y_train, open('../data/imdb_sequence/1000_one_hot/y_tr_seq_set.pkl','wb'))

# # pickle.dump(X_tr_encoder_input, open('../data/imdb_sequence/1000_one_hot/X_tr_encoder.pkl','wb'))
# # pickle.dump(y_tr_decoder , open('../data/imdb_sequence/1000_one_hot/y_tr_decoder.pkl','wb'))


# pickle.dump(X_test, open('../data/imdb_sequence/1000_one_hot/X_te_seq_set.pkl','wb'))
# pickle.dump(y_test, open('../data/imdb_sequence/1000_one_hot/y_te_seq_set.pkl','wb'))

### GloVe

In [3]:
GLOVE_DIR = "../data/glove.6B/"
GLOVE_DIM = 100

def extract_glove_index(file):
    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, file), 'r')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

embeddings_index = extract_glove_index('glove.6B.100d.txt')
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [16]:
embeddings_index['.']

array([-0.33979  ,  0.20941  ,  0.46348  , -0.64792  , -0.38377  ,
        0.038034 ,  0.17127  ,  0.15978  ,  0.46619  , -0.019169 ,
        0.41479  , -0.34349  ,  0.26872  ,  0.04464  ,  0.42131  ,
       -0.41032  ,  0.15459  ,  0.022239 , -0.64653  ,  0.25256  ,
        0.043136 , -0.19445  ,  0.46516  ,  0.45651  ,  0.68588  ,
        0.091295 ,  0.21875  , -0.70351  ,  0.16785  , -0.35079  ,
       -0.12634  ,  0.66384  , -0.2582   ,  0.036542 , -0.13605  ,
        0.40253  ,  0.14289  ,  0.38132  , -0.12283  , -0.45886  ,
       -0.25282  , -0.30432  , -0.11215  , -0.26182  , -0.22482  ,
       -0.44554  ,  0.2991   , -0.85612  , -0.14503  , -0.49086  ,
        0.0082973, -0.17491  ,  0.27524  ,  1.4401   , -0.21239  ,
       -2.8435   , -0.27958  , -0.45722  ,  1.6386   ,  0.78808  ,
       -0.55262  ,  0.65     ,  0.086426 ,  0.39012  ,  1.0632   ,
       -0.35379  ,  0.48328  ,  0.346    ,  0.84174  ,  0.098707 ,
       -0.24213  , -0.27053  ,  0.045287 , -0.40147  ,  0.1139

In [23]:
# embeddings_index.keys()

# embeddings_index['n\'t']
embeddings_index['<UNK>']

KeyError: '<UNK>'

In [20]:
embeddings_index['not']

array([-1.9104e-01,  1.7601e-01,  3.6920e-01, -5.0323e-01, -4.7561e-01,
        1.5798e-01, -1.1679e-01,  2.1052e-01,  3.2652e-01,  1.2194e-01,
        9.0944e-02,  2.6089e-01,  7.6294e-01,  6.9673e-04, -5.0001e-02,
       -4.4853e-01,  3.6239e-01,  5.6345e-01, -6.8702e-01,  3.3237e-01,
        3.1285e-01, -1.4207e-01,  3.5327e-01, -1.6426e-01, -1.0693e-01,
        7.7786e-02, -1.7704e-01, -9.2897e-01,  1.4680e-01, -1.3585e-01,
        2.5682e-01,  6.6019e-01, -3.5569e-01,  2.1838e-01,  3.8173e-01,
        5.4337e-01,  1.0197e-01,  3.5230e-01, -2.5510e-01, -1.5155e-01,
       -6.7434e-01,  1.6903e-01,  1.6413e-01, -5.3843e-01, -1.7457e-01,
       -2.8539e-01,  7.4044e-01, -6.7533e-01, -2.3382e-01, -1.3599e+00,
        3.0225e-01, -1.4968e-01,  2.7043e-01,  1.1979e+00, -2.9556e-01,
       -2.5395e+00,  1.0303e-03, -2.6272e-01,  1.8303e+00,  8.0008e-01,
       -3.5691e-01,  5.6578e-01, -5.5040e-01,  7.0845e-02,  1.4275e+00,
        9.0160e-02,  7.8420e-01,  7.8490e-01, -3.3538e-01, -6.57

In [17]:
max_encoder_seq_length = 81
max_decoder_seq_length = 5

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_sequence)

X_tr_padded = pad_sequences(tokenizer.texts_to_sequences(X_train_sequence), maxlen=81, padding='post', truncating='post')
y_tr_padded = pad_sequences(tokenizer.texts_to_sequences(y_train_target), maxlen=5, padding='post', truncating='post')



In [23]:
encoder_input_data = np.zeros(
    (len(X_train_sequence), max_encoder_seq_length, GLOVE_DIM),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(y_train_target), max_decoder_seq_length, GLOVE_DIM),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(y_train_target), max_decoder_seq_length, len(tokenizer.word_index)),
    dtype='float32')

In [42]:
train_sequence = []
target_sequence = []

for sample in X_train_sequence:
    train_sequence.append(sample.split())
for target in y_train_target:
    target_sequence.append(target.split())

In [43]:
seqlen = [len(seq) for seq in target_sequence]
seqlen = np.asarray(seqlen)
np.max(seqlen)

5

In [41]:
seqlen

array([70, 70, 70, ..., 70, 70, 70])

In [44]:
# padding

padding_index = (seqlen < 81).astype(int)
a = [1,2,3,4]

a.insert(0, '<pad>')
a.insert(0, 0)
a

[0, '<pad>', 1, 2, 3, 4]

In [29]:
np.max(y_tr_padded)

42356

In [44]:
# 100-dim -> input sequence, input decoder
# 42K-dim -> output sequence.

for i, (input_text, target_text, target_padded) in enumerate(zip(train_sequence, target_sequence, y_tr_padded)):
    for t, word in enumerate(input_text):
        try:
            encoder_input_data[i, t, :] = embeddings_index[word]
        except KeyError as error:
            continue
    
    for t, word in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        try:
            decoder_input_data[i, t, :] = embeddings_index[word]
        except KeyError as error:
            continue
        
    for t, word in enumerate(target_padded):
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, word] = 1.
    

In [22]:
len(embeddings_index.keys())

400000

In [21]:
decoder_target_data.shape

(22752, 81, 1000)

In [12]:
words = list(t.word_docs.keys())

In [13]:
max_encoder_seq_length = X_tr_padded.shape[1]
max_decoder_seq_length = y_tr_padded.shape[1]


In [14]:
print('Number of samples:', len(X_tr_sequence))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 22752
Number of unique input tokens: 3000
Number of unique output tokens: 3000
Max sequence length for inputs: 85
Max sequence length for outputs: 5


In [15]:
y_tr_padded.shape

(22752, 5)

### Sample Code

## Encoder - decoder here

In [20]:
encoder_input_data = np.zeros(
    (len(X_tr_sequence), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(y_tr_sequence), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(y_tr_sequence), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


for i, (input_text, target_text) in enumerate(zip(X_tr_padded, y_tr_padded)):
    for t, word in enumerate(input_text):
        encoder_input_data[i, t, word] = 1.
        
    for t, word in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, word] = 1.
        
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, word] = 1.

In [21]:
latent_dim = 100

In [22]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [23]:
epochs = 500
batch_size = 128

In [24]:

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2)

Train on 18201 samples, validate on 4551 samples
Epoch 1/500
18201/18201 [==============================] - 61s 3ms/step - loss: 4.7087 - val_loss: 4.2728
Epoch 2/500
18201/18201 [==============================] - 54s 3ms/step - loss: 4.1975 - val_loss: 4.1529
Epoch 3/500
18201/18201 [==============================] - 54s 3ms/step - loss: 4.0842 - val_loss: 4.0701
Epoch 4/500
18201/18201 [==============================] - 54s 3ms/step - loss: 4.0066 - val_loss: 4.0062
Epoch 5/500
18201/18201 [==============================] - 54s 3ms/step - loss: 3.9520 - val_loss: 3.9620
Epoch 6/500
18201/18201 [==============================] - 54s 3ms/step - loss: 3.9069 - val_loss: 3.9292
Epoch 7/500
18201/18201 [==============================] - 54s 3ms/step - loss: 3.8667 - val_loss: 3.8924
Epoch 8/500
18201/18201 [==============================] - 54s 3ms/step - loss: 3.8275 - val_loss: 3.8614
Epoch 9/500
18201/18201 [==============================] - 54s 3ms/step - loss: 3.7850 - val_loss: 3.82

18201/18201 [==============================] - 55s 3ms/step - loss: 2.6873 - val_loss: 3.6039
Epoch 154/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6849 - val_loss: 3.6116
Epoch 155/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6827 - val_loss: 3.6133
Epoch 156/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6788 - val_loss: 3.6179
Epoch 157/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6778 - val_loss: 3.6197
Epoch 158/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6753 - val_loss: 3.6245
Epoch 159/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6727 - val_loss: 3.6306
Epoch 160/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6692 - val_loss: 3.6377
Epoch 161/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.6720 - val_loss: 3.6333
Epoch 162/500
18201/18201 [===============

Epoch 229/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4968 - val_loss: 3.8519
Epoch 230/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4931 - val_loss: 3.8592
Epoch 231/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4910 - val_loss: 3.8655
Epoch 232/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4885 - val_loss: 3.8567
Epoch 233/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4857 - val_loss: 3.8674
Epoch 234/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4824 - val_loss: 3.8738
Epoch 235/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4774 - val_loss: 3.8750
Epoch 236/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4757 - val_loss: 3.8820
Epoch 237/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.4751 - val_loss: 3.8865
Epoch 238/500
18201/18201 [=

Epoch 305/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2807 - val_loss: 4.1385
Epoch 306/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2763 - val_loss: 4.1447
Epoch 307/500
18201/18201 [==============================] - 55s 3ms/step - loss: 2.2734 - val_loss: 4.1503
Epoch 308/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2705 - val_loss: 4.1477
Epoch 309/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2687 - val_loss: 4.1510
Epoch 310/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2669 - val_loss: 4.1586
Epoch 311/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2634 - val_loss: 4.1671
Epoch 312/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2618 - val_loss: 4.1748
Epoch 313/500
18201/18201 [==============================] - 54s 3ms/step - loss: 2.2565 - val_loss: 4.1711
Epoch 314/500
18201/18201 [=

Epoch 381/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0941 - val_loss: 4.4449
Epoch 382/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0884 - val_loss: 4.4494
Epoch 383/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0814 - val_loss: 4.4559
Epoch 384/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0830 - val_loss: 4.4596
Epoch 385/500
18201/18201 [==============================] - 75s 4ms/step - loss: 2.0807 - val_loss: 4.4629
Epoch 386/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0750 - val_loss: 4.4682
Epoch 387/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0725 - val_loss: 4.4735
Epoch 388/500
18201/18201 [==============================] - 75s 4ms/step - loss: 2.0705 - val_loss: 4.4749
Epoch 389/500
18201/18201 [==============================] - 74s 4ms/step - loss: 2.0722 - val_loss: 4.4746
Epoch 390/500
18201/18201 [=

Epoch 457/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8786 - val_loss: 4.7150
Epoch 458/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8764 - val_loss: 4.7290
Epoch 459/500
18201/18201 [==============================] - 75s 4ms/step - loss: 1.8777 - val_loss: 4.7303
Epoch 460/500
18201/18201 [==============================] - 75s 4ms/step - loss: 1.8725 - val_loss: 4.7270
Epoch 461/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8662 - val_loss: 4.7264
Epoch 462/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8588 - val_loss: 4.7391
Epoch 463/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8610 - val_loss: 4.7419
Epoch 464/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8581 - val_loss: 4.7483
Epoch 465/500
18201/18201 [==============================] - 74s 4ms/step - loss: 1.8594 - val_loss: 4.7438
Epoch 466/500
18201/18201 [=

In [53]:
# model.save('s2s.h5')

/home/anneke/.local/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 100) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 3000)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 3000)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100), (None, 1240400     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 100),  1240400     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

### Inference

In [36]:
# sampling models

# https://nlp.stanford.edu/~johnhew/public/14-seq2seq.pdf
# https://medium.com/machine-learning-bites/deeplearning-series-sequence-to-sequence-architectures-4c4ca89e5654

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                     [decoder_outputs] + decoder_states)

In [54]:
# encoder_model.save('encoder.h5')
# decoder_model.save('decoder.h5')

/home/anneke/.local/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_5:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'input_6:0' shape=(?, 100) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [37]:
reverse_word_index = dict((i,word) for word,i in word_index.items())

In [38]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    
#     target_seq = np.random.rand(1,1, num_decoder_tokens)
    target_seq = np.zeros((1,1, num_decoder_tokens))
    
    decoded_sentence = ''
    
    for i in range(5):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         print(np.max(output_tokens))
#         print(sorted(output_tokens[0, -1, :])[-5:])
        if sampled_token_index == 0:
            sampled_word = ' '
        else:
            sampled_word = reverse_word_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word
        
        target_seq = np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index] = 1.
        
        states_value = [h, c]
        
    return decoded_sentence

In [51]:
def decode_sequence_target(input_seq, true_target_seq):
    states_value = encoder_model.predict(input_seq)
    true_states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1, num_decoder_tokens))
    from_true_target_seq = np.zeros((1,1, num_decoder_tokens))
    true_token_index = true_target_seq[0,0]
    target_seq[0,0,true_token_index] = 1.
    
    decoded_sentence = ''
    if true_token_index == 0:
            sampled_word = ' '
    else:
            sampled_word = reverse_word_index[true_token_index]
            
    decoded_sentence += ' ' + sampled_word
    
    true_joint_log_prob = 0
    pred_joint_log_prob = 0
    
    for i in range(4):
        # from prediction
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # from true target
        from_true_output_tokens, h_true, c_true = decoder_model.predict([from_true_target_seq] + true_states_value)
        
        predict_target_prob = np.max(output_tokens[0, -1, :])
        true_target_prob = from_true_output_tokens[0,-1, true_target_seq[0,i]]
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        true_token_index = true_target_seq[0,i]
        
        true_joint_log_prob += np.log(true_target_prob)
        pred_joint_log_prob += np.log(predict_target_prob)
        
        if sampled_token_index == 0:
            sampled_word = ' '
        else:
            sampled_word = reverse_word_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word
        
        target_seq = np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index] = 1.
        
        states_value = [h, c]
        
        from_true_target_seq = np.zeros((1,1,num_decoder_tokens))
        from_true_target_seq[0,0,true_token_index] = 1.
        
        true_states_value = [h_true, c_true]
    
#     print(true_joint_log_prob, pred_joint_log_prob)
    
    # print the last prob
#     print(true_target_prob, predict_target_prob)
    return decoded_sentence

In [52]:
for seq_index in range(80,100):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    true_target_seq = y_tr_padded[seq_index:seq_index+1]
    decoded_sentence = decode_sequence_target(input_seq, true_target_seq)
    
#     print('Input sentence:', X_train_sequence[seq_index])
    print()
    print('Output sentence:', decoded_sentence)
    print('True Target sentence:', y_train_target[seq_index])
    print()


Output sentence:  far the best movie ever
True Target sentence: far the best show on


Output sentence:  a total waste of time
True Target sentence: a total waste of time


Output sentence:  imagery how beautifully a making
True Target sentence: imagery how beautifully the opening


Output sentence:    was best described as
True Target sentence: borel is unfortunately not a


Output sentence:  of the best performance in
True Target sentence: of the worst film i


Output sentence:    the best known for
True Target sentence: the always fantastic kevin spacey


Output sentence:    and the excellent story
True Target sentence: was so beautiful then


Output sentence:  do not waste your time
True Target sentence: do not waste your time


Output sentence:    and the fantastic of
True Target sentence: is my favorite of vincent


Output sentence:  during the best of the
True Target sentence: during the worst terrorist attack


Output sentence:      the best enjoyed
True Target sentence: singl

## Play with Candidate Sequence

### Without the predicted prob

In [67]:
# Play with candidate

def decode_sequence_target(input_seq, candidate_target_seq):
    candidate_states_value = encoder_model.predict(input_seq)

    from_candidate_target_seq = np.zeros((1,1, num_decoder_tokens))
    candidate_token_index = candidate_target_seq[0,0]
    from_candidate_target_seq[0,0,candidate_token_index] = 1.
    candidate_joint_log_prob = 0
    
    for i in range(4):
        from_candidate_output_tokens, h_true, c_true = decoder_model.predict([from_candidate_target_seq] + candidate_states_value)
    
        candidate_target_prob = from_candidate_output_tokens[0,-1, candidate_target_seq[0,i]]
        candidate_token_index = candidate_target_seq[0,i]
        candidate_joint_log_prob += np.log(candidate_target_prob)
        
        # get the t+1 input
        from_candidate_target_seq = np.zeros((1,1,num_decoder_tokens))
        from_candidate_target_seq[0,0,candidate_token_index] = 1.
        
        candidate_states_value = [h_true, c_true]

    return candidate_joint_log_prob, candidate_target_prob

In [74]:
seq_index = 0

candidate_jll = []
sliding_y_tr = []

for i in range(X_tr_padded[seq_index].shape[0]-1-5):
    sliding_y_tr.append(X_tr_padded[0, i:i+5])

sliding_y_tr = np.asarray(sliding_y_tr)

print('Input sentence:', X_train_sequence[seq_index], '\n')

for i in range(sliding_y_tr.shape[0]):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    candidate_seq = sliding_y_tr[i:i+1]
    candidate_jll_slide, candidate_last_prob = decode_sequence_target(input_seq, candidate_seq)
    
    candidate_jll.append([candidate_jll_slide, candidate_last_prob])
    print('true sequence:', y_tr_sequence[0])
    print('Candidate sequence:', candidate_seq)
    print('%d\t%.7f \t %.7f' %(i+1, candidate_jll_slide, candidate_last_prob))
    print()

Input sentence: even kind of a happy ending of sort whee a step up from part 4 but not much of one again brian yuzna is involved and screaming mad george so some decent special effect but not enough to make this great a few leftover from part 4 are hanging around too like clint howard and neith hunter but that does not really make any difference anyway i now have seeing the whole series out of my system now if i could 

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[0 0 0 0 0]]
1	-98.5723982 	 0.0000072

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[0 0 0 0 0]]
2	-98.5723982 	 0.0000072

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[0 0 0 0 0]]
3	-98.5723982 	 0.0000072

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[0 0 0 0 0]]
4	-98.5723982 	 0.0000072

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[0 0 0 0 0]]
5	-98.5723982 	 0.0000072

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[ 0  0  0  0 60]]
6	-98.5723982 	 

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[202 184  50   5  57]]
76	-39.0628848 	 0.0021762

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[ 184   50    5   57 1501]]
77	-41.6054320 	 0.0003778

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[  50    5   57 1501  160]]
78	-51.3717451 	 0.0019707

true sequence: [98, 10, 56, 3, 167]
Candidate sequence: [[   5   57 1501  160   43]]
79	-61.6393394 	 0.0000009



In [89]:
decode_slide = []
for i in sliding_y_tr:
    decode_sentence = ''
    for j in i:
        if j == 0:
            word = ' '
        else:
            word = reverse_word_index[j]
        decode_sentence += word + ' '
    decode_slide.append(decode_sentence)
    
decode_slide

['          ',
 '          ',
 '          ',
 '          ',
 '          ',
 '        even ',
 '      even kind ',
 '    even kind of ',
 '  even kind of a ',
 'even kind of a happy ',
 'kind of a happy ending ',
 'of a happy ending of ',
 'a happy ending of sort ',
 'happy ending of sort a ',
 'ending of sort a step ',
 'of sort a step up ',
 'sort a step up from ',
 'a step up from part ',
 'step up from part 4 ',
 'up from part 4 but ',
 'from part 4 but not ',
 'part 4 but not much ',
 '4 but not much of ',
 'but not much of one ',
 'not much of one again ',
 'much of one again brian ',
 'of one again brian is ',
 'one again brian is involved ',
 'again brian is involved and ',
 'brian is involved and screaming ',
 'is involved and screaming mad ',
 'involved and screaming mad george ',
 'and screaming mad george so ',
 'screaming mad george so some ',
 'mad george so some decent ',
 'george so some decent special ',
 'so some decent special effect ',
 'some decent special effect bu

In [88]:
decode_slide[46]

# we can calculate how far it is from the target sequence in terms of index

' this great a few from'

In [79]:
candidate_jll = np.asarray(candidate_jll)

np.max(candidate_jll[:,0]), np.argmax(candidate_jll[:,0])

(-20.44260811805725, 46)

In [80]:
sliding_y_tr[46]

array([ 10,  56,   3, 167,  36], dtype=int32)

In [81]:
sliding_y_tr

array([[   0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0],
       [   0,    0,    0,    0,   60],
       [   0,    0,    0,   60,  241],
       [   0,    0,   60,  241,    5],
       [   0,   60,  241,    5,    3],
       [  60,  241,    5,    3,  716],
       [ 241,    5,    3,  716,  258],
       [   5,    3,  716,  258,    5],
       [   3,  716,  258,    5,  439],
       [ 716,  258,    5,  439,    3],
       [ 258,    5,  439,    3, 1087],
       [   5,  439,    3, 1087,   65],
       [ 439,    3, 1087,   65,   36],
       [   3, 1087,   65,   36,  129],
       [1087,   65,   36,  129,  408],
       [  65,   36,  129,  408,   17],
       [  36,  129,  408,   17,   12],
       [ 129,  408,   17,   12,   79],
       [ 408,   17,   12,   79,    5],
       [  17,   12,   79,    5,   24],
       [  12,   79,    5,   24,  181],
       [  79,    5,   24,

### For all

In [ ]:
def get_jll()

In [93]:
def generate_candidate_list(X):
    y_candidate = []
    
    for i in range(X.shape[0]-1-5):
        y_candidate.append(X[i:i+5])
    
    return np.asarray(y_candidate)

In [179]:
file = open("jll.csv", "a")
start = 2198
i = 2198

In [180]:
for doc in X_tr_padded[start:]:
    y_candidate = generate_candidate_list(doc)
    
    candidate_jll_per_doc = []
    input_seq = encoder_input_data[i:i+1]
    
    true_target_index = target_index(i, y_candidate, y_tr_sequence[i])
    
    # Encode
    candidate_states_value = encoder_model.predict(input_seq)
    
    for j in range(y_candidate.shape[0]):
        candidate_seq = y_candidate[j:j+1]
        candidate_jll_slide, candidate_last_prob = decode_sequence_target(candidate_states_value, candidate_seq)
        candidate_jll_per_doc.append(candidate_jll_slide)

    candidate_jll_per_doc = np.asarray(candidate_jll_per_doc)
    max_jll_index = np.argmax(candidate_jll_per_doc)
    true_target_jll = np.around(candidate_jll_per_doc[true_target_index],5)
    max_candidate_jll = np.around(candidate_jll_per_doc[max_jll_index],5)
    
    
    file.write('%d\t%d\t%s\t%d\t%s\t%d\t%.5f\t%.5f\t%.5f\t%d\n' %(i, true_target_index, y_train_target[i],
                                                            max_jll_index, to_sequence(y_candidate[max_jll_index]),
                                                            -(true_target_index-max_jll_index),
                                                            true_target_jll, max_candidate_jll,
                                                            np.absolute(true_target_jll-max_candidate_jll),
                                                            len(intersection(y_tr_sequence[i], y_candidate[max_jll_index]))))
    if i % 100 == 0:
        print('Processing document %d...' %(i))
        
    i += 1
    
file.close()

Processing document 2200...


/home/anneke/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


Processing document 2300...
Processing document 2400...
Processing document 2500...
Processing document 2600...
Processing document 2700...
Processing document 2800...
Processing document 2900...
Processing document 3000...
Processing document 3100...
Processing document 3200...
Processing document 3300...
Processing document 3400...
Processing document 3500...
Processing document 3600...
Processing document 3700...
Processing document 3800...
Processing document 3900...
Processing document 4000...
Processing document 4100...
Processing document 4200...
Processing document 4300...
Processing document 4400...
Processing document 4500...
Processing document 4600...
Processing document 4700...
Processing document 4800...
Processing document 4900...
Processing document 5000...
Processing document 5100...
Processing document 5200...
Processing document 5300...
Processing document 5400...
Processing document 5500...
Processing document 5600...
Processing document 5700...
Processing document 

KeyboardInterrupt: 

In [181]:
file.close()

In [160]:
i

2198

In [111]:
# doc num, doc index argmax

def to_sequence(int_sequence):
    decoded = ''
    for i in int_sequence:
        if i == 0:
            word = ' '
        else:
            word = reverse_word_index[i]
        decoded += word + ' '
    return decoded


In [139]:
# Play with candidate

def decode_sequence_target(candidate_states_value, candidate_target_seq):
#     candidate_states_value = encoder_model.predict(input_seq)

    from_candidate_target_seq = np.zeros((1,1, num_decoder_tokens))
    candidate_token_index = candidate_target_seq[0,0]
    from_candidate_target_seq[0,0,candidate_token_index] = 1.
    candidate_joint_log_prob = 0
    
    for i in range(4):
        from_candidate_output_tokens, h_true, c_true = decoder_model.predict([from_candidate_target_seq] + candidate_states_value)
    
        candidate_target_prob = from_candidate_output_tokens[0,-1, candidate_target_seq[0,i]]
        candidate_token_index = candidate_target_seq[0,i]
        candidate_joint_log_prob += np.log(candidate_target_prob)
        
        # get the t+1 input
        from_candidate_target_seq = np.zeros((1,1,num_decoder_tokens))
        from_candidate_target_seq[0,0,candidate_token_index] = 1.
        
        candidate_states_value = [h_true, c_true]

    return candidate_joint_log_prob, candidate_target_prob

In [125]:
y_tr_sequence[3]

[49, 1, 35, 5, 10]

In [137]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def target_index(doc_idx, candidate_seq, y):
    for i,j in enumerate(candidate_seq):
        if len(intersection(j, y)) == len(y):
            return i
    return -1

In [136]:
for i,j in enumerate(y_candidate):
    if len(intersection(j, y_tr_sequence[0])) == len(y_tr_sequence[0]):
        print(i)

45
